In [1]:
from Frame import Frame
from Camera import Camera
import scipy
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)
import numpy as np
from fly_model import Plotter
import plotly.graph_objects as go
import Utils
path = 'G:/My Drive/Research/gaussian_splatting/gaussian_splatting_input/mov19_2022_03_03/'


frames = list(range(900,910,1))

image_name= []
for frame in frames:
    image_name += [f'P{frame}CAM{cam + 1}' for cam in range(4)]
cameras = {f'cam{cam + 1}':Camera(path,cam) for cam in range(4)}
frames = {f'{im_name}.jpg':Frame(path,im_name,idx) for idx,im_name in enumerate(image_name)}

In [15]:
frame_number = 900 
frame_names = ['P900CAM1.jpg','P900CAM2.jpg','P900CAM3.jpg','P900CAM4.jpg']
frame_names = [f'P{frame_number}CAM{idx}.jpg' for idx in range(1,5)]

ray_world = np.vstack([frames[frame].camera_center_to_pixel_ray(frames[frame].cm) for frame in  frame_names])
camera_center = np.vstack([frames[frame].X0.T for frame in  frame_names])
point = Utils.triangulate_least_square(camera_center,ray_world)





In [16]:


marker_dict = {'size': 5, 'color': 'red'}
marker_dict_point = {'size': 5, 'color': 'blue'}

line_dict = {'width': 3, 'color': 'red'}
fig = go.Figure()

[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'markers',marker_dict = marker_dict) for origin,end,idx in zip(camera_center, ray_world,range(4))]
[Plotter.scatter3d(fig,np.vstack((origin,end)),idx, mode = 'lines',line_dict = line_dict) for origin,end,idx in zip(camera_center, ray_world,range(4))]
Plotter.scatter3d(fig,point[:,np.newaxis].T,'least square', mode = 'markers',marker_dict = marker_dict_point) 

fig.show()